# 뉴스 기사 분류: 다중 분류 문제

## 추가 실험

* 더 크거나 작은 층을 사용해 보세요: 32개 유닛, 128개 유닛 등 (교재에서는 64, 64, 46 썼음. 출력층은 46으로 고정)
* 교재의 예시에서는 두 개의 은닉층을 사용했습니다. 한 개의 은닉층이나 세 개의 은닉층을 사용해 보세요.

In [1]:
import keras
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

C:\Users\leehyunjoo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
import numpy as np
def vectorize_sequences(sequences, dimenstion=10000):
    results = np.zeros((len(sequences), dimenstion))
    for i, sequences in enumerate(sequences):
        results[i,sequences] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

영화 이진 분류에서는 레이블이 0, 1 로 2개였는데, 이번 뉴스 기사 분류는 레이블이 46개이다.

In [8]:
# 레이블을 벡터로 변환하는 방법1)
# 방법1 말고 방법2 실행할 것임 (어짜피 해도 똑같긴함)
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

In [9]:
# 레이블을 벡터로 변환하는 방법2) (그냥 똑같으니까 실행해도 상관없음)
from keras.utils.np_utils import to_categorical
one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

In [12]:
from keras import models
from keras import layers

# 기본 예제
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# 컴파일을 한 후에 훈련을 검증해보는 것이다.
# 훈련 데이터 (train)에서 1000개의 샘플을 떼어내서 검증(validation) 세트로 사용한다.
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]


history = model.fit(partial_x_train,partial_y_train,
                    epochs=20, batch_size=512, validation_data=(x_val, y_val))

Train on 7982 samples, validate on 1000 samples
Epoch 1/20
7982/7982 [==============================] - 1s 118us/step - loss: 2.6213 - accuracy: 0.5481 - val_loss: 1.7195 - val_accuracy: 0.6560
Epoch 2/20
7982/7982 [==============================] - 1s 101us/step - loss: 1.3986 - accuracy: 0.7161 - val_loss: 1.3020 - val_accuracy: 0.7220
Epoch 3/20
7982/7982 [==============================] - 1s 97us/step - loss: 1.0257 - accuracy: 0.7845 - val_loss: 1.1370 - val_accuracy: 0.7550
Epoch 4/20
7982/7982 [==============================] - 1s 97us/step - loss: 0.8087 - accuracy: 0.8297 - val_loss: 1.0534 - val_accuracy: 0.7640
Epoch 5/20
7982/7982 [==============================] - 1s 96us/step - loss: 0.6483 - accuracy: 0.8642 - val_loss: 0.9864 - val_accuracy: 0.7900
Epoch 6/20
7982/7982 [==============================] - 1s 104us/step - loss: 0.5182 - accuracy: 0.8936 - val_loss: 0.9463 - val_accuracy: 0.8000
Epoch 7/20
7982/7982 [==============================] - 1s 98us/step - loss: 0.

16번하면 역전되니까 16로 하자.

In [ ]:
# 추가 문제 : 더 작은 층을 사용해보자, 1개의 은닉층만 사용해보자.
# 마지막 Dense 층이 46인 것은, 각 입력 샘플에 대해 46차원의 벡터를 출력한다는 뜻
# softmax 함수는, 46일때, 46차원의 출력벡터를 만들고,
# output[i] 는 어떤 샘플이 클래스 i에 속할 확률임. 46개의 값을 다 더하면 1 이다.

# 영화 이진 분류에서는 16차원을 사용했었는데, 그때는 출력 클래스의 종류가 2개였다.
# 그런데 여기서는 클래스가 46개이기 때문에, 16차원으로는 충분하지 않을 것 같아. 그래서 64 차원을 사용했다.
# Q. 차원은 16의 배수로 하는 것인가?

In [14]:
# 모델 처음부터 다시 훈련
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# 아까 처음에 훈련할때는 일부만 model.fit(partial_x_train, partial_y_train) 하고
# 과대적합 어딘지 찾은 다음에는 전체인 (x_train, one_hot_train_labels)으로 하는 것이다!!

model.fit(x_train, one_hot_train_labels,
          epochs=16, batch_size=512, validation_data=(x_val, y_val))
results = model.evaluate(x_test, one_hot_test_labels)

Train on 8982 samples, validate on 1000 samples
Epoch 1/16
8982/8982 [==============================] - 1s 113us/step - loss: 2.5583 - accuracy: 0.5638 - val_loss: 1.5117 - val_accuracy: 0.6930
Epoch 2/16
8982/8982 [==============================] - 1s 99us/step - loss: 1.3403 - accuracy: 0.7168 - val_loss: 1.0404 - val_accuracy: 0.7640
Epoch 3/16
8982/8982 [==============================] - 1s 97us/step - loss: 0.9973 - accuracy: 0.7826 - val_loss: 0.7828 - val_accuracy: 0.8410
Epoch 4/16
8982/8982 [==============================] - 1s 96us/step - loss: 0.7796 - accuracy: 0.8329 - val_loss: 0.5899 - val_accuracy: 0.8810
Epoch 5/16
8982/8982 [==============================] - 1s 98us/step - loss: 0.6145 - accuracy: 0.8684 - val_loss: 0.4572 - val_accuracy: 0.9080
Epoch 6/16
8982/8982 [==============================] - 1s 98us/step - loss: 0.4864 - accuracy: 0.8958 - val_loss: 0.3556 - val_accuracy: 0.9300
Epoch 7/16
8982/8982 [==============================] - 1s 101us/step - loss: 0.3

In [15]:
results

[1.0760353256949122, 0.8000890612602234]

대략 80%의 정확도 달성.

In [16]:
# 추가 문제 (은닉층 하나로)
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
# model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# 컴파일을 한 후에 훈련을 검증해보는 것이다.
# 훈련 데이터 (train)에서 1000개의 샘플을 떼어내서 검증(validation) 세트로 사용한다.
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]


history = model.fit(partial_x_train,partial_y_train,
                    epochs=20, batch_size=512, validation_data=(x_val, y_val))



Train on 7982 samples, validate on 1000 samples
Epoch 1/20
7982/7982 [==============================] - 1s 109us/step - loss: 2.5451 - accuracy: 0.5564 - val_loss: 1.7853 - val_accuracy: 0.6580
Epoch 2/20
7982/7982 [==============================] - 1s 99us/step - loss: 1.4495 - accuracy: 0.7200 - val_loss: 1.3378 - val_accuracy: 0.7220
Epoch 3/20
7982/7982 [==============================] - 1s 104us/step - loss: 1.0645 - accuracy: 0.7876 - val_loss: 1.1384 - val_accuracy: 0.7630
Epoch 4/20
7982/7982 [==============================] - 1s 97us/step - loss: 0.8341 - accuracy: 0.8335 - val_loss: 1.0197 - val_accuracy: 0.7890
Epoch 5/20
7982/7982 [==============================] - 1s 97us/step - loss: 0.6710 - accuracy: 0.8672 - val_loss: 0.9448 - val_accuracy: 0.8040
Epoch 6/20
7982/7982 [==============================] - 1s 100us/step - loss: 0.5485 - accuracy: 0.8914 - val_loss: 0.8929 - val_accuracy: 0.8140
Epoch 7/20
7982/7982 [==============================] - 1s 99us/step - loss: 0.

In [17]:
# 모델 처음부터 다시 훈련
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
# model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# 아까 처음에 훈련할때는 일부만 model.fit(partial_x_train, partial_y_train) 하고
# 과대적합 어딘지 찾은 다음에는 전체인 (x_train, one_hot_train_labels)으로 하는 것이다!!

model.fit(x_train, one_hot_train_labels,
          epochs=16, batch_size=512, validation_data=(x_val, y_val))
results = model.evaluate(x_test, one_hot_test_labels)

Train on 8982 samples, validate on 1000 samples
Epoch 1/16
8982/8982 [==============================] - 1s 104us/step - loss: 2.5311 - accuracy: 0.5354 - val_loss: 1.6455 - val_accuracy: 0.6810
Epoch 2/16
8982/8982 [==============================] - 1s 97us/step - loss: 1.4319 - accuracy: 0.7211 - val_loss: 1.0867 - val_accuracy: 0.7960
Epoch 3/16
8982/8982 [==============================] - 1s 97us/step - loss: 1.0293 - accuracy: 0.7923 - val_loss: 0.8029 - val_accuracy: 0.8480
Epoch 4/16
8982/8982 [==============================] - 1s 97us/step - loss: 0.7949 - accuracy: 0.8395 - val_loss: 0.6191 - val_accuracy: 0.8860
Epoch 5/16
8982/8982 [==============================] - 1s 98us/step - loss: 0.6355 - accuracy: 0.8749 - val_loss: 0.4907 - val_accuracy: 0.9130
Epoch 6/16
8982/8982 [==============================] - 1s 96us/step - loss: 0.5161 - accuracy: 0.8987 - val_loss: 0.3997 - val_accuracy: 0.9280
Epoch 7/16
8982/8982 [==============================] - 1s 98us/step - loss: 0.42

In [18]:
results

[0.9396783148509948, 0.8076580762863159]

이것도 80프로의 정확도

In [19]:
# 추가 문제 (32개의 유닛)
# 46차원보다 32가 더 작으니까 결과 더 안좋게 나오지 않을까..?
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# 컴파일을 한 후에 훈련을 검증해보는 것이다.
# 훈련 데이터 (train)에서 1000개의 샘플을 떼어내서 검증(validation) 세트로 사용한다.
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]


history = model.fit(partial_x_train,partial_y_train,
                    epochs=20, batch_size=512, validation_data=(x_val, y_val))

Train on 7982 samples, validate on 1000 samples
Epoch 1/20
7982/7982 [==============================] - 1s 105us/step - loss: 3.0894 - accuracy: 0.4678 - val_loss: 2.3786 - val_accuracy: 0.5970
Epoch 2/20
7982/7982 [==============================] - 1s 89us/step - loss: 1.9733 - accuracy: 0.6495 - val_loss: 1.7110 - val_accuracy: 0.6560
Epoch 3/20
7982/7982 [==============================] - 1s 86us/step - loss: 1.4479 - accuracy: 0.7091 - val_loss: 1.4022 - val_accuracy: 0.6990
Epoch 4/20
7982/7982 [==============================] - 1s 89us/step - loss: 1.1650 - accuracy: 0.7585 - val_loss: 1.2358 - val_accuracy: 0.7360
Epoch 5/20
7982/7982 [==============================] - 1s 92us/step - loss: 0.9751 - accuracy: 0.7980 - val_loss: 1.1336 - val_accuracy: 0.7560
Epoch 6/20
7982/7982 [==============================] - 1s 89us/step - loss: 0.8316 - accuracy: 0.8252 - val_loss: 1.0630 - val_accuracy: 0.7700
Epoch 7/20
7982/7982 [==============================] - 1s 89us/step - loss: 0.71

18로 하자

In [21]:
# 모델 처음부터 다시 훈련
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# 아까 처음에 훈련할때는 일부만 model.fit(partial_x_train, partial_y_train) 하고
# 과대적합 어딘지 찾은 다음에는 전체인 (x_train, one_hot_train_labels)으로 하는 것이다!!

model.fit(x_train, one_hot_train_labels,
          epochs=18, batch_size=512, validation_data=(x_val, y_val))
results = model.evaluate(x_test, one_hot_test_labels)

Train on 8982 samples, validate on 1000 samples
Epoch 1/18
8982/8982 [==============================] - 1s 107us/step - loss: 3.1757 - accuracy: 0.4463 - val_loss: 2.3822 - val_accuracy: 0.5990
Epoch 2/18
8982/8982 [==============================] - 1s 89us/step - loss: 2.0008 - accuracy: 0.6238 - val_loss: 1.5989 - val_accuracy: 0.6640
Epoch 3/18
8982/8982 [==============================] - 1s 90us/step - loss: 1.4572 - accuracy: 0.6902 - val_loss: 1.2299 - val_accuracy: 0.7310
Epoch 4/18
8982/8982 [==============================] - 1s 91us/step - loss: 1.1771 - accuracy: 0.7482 - val_loss: 1.0022 - val_accuracy: 0.7950
Epoch 5/18
8982/8982 [==============================] - 1s 92us/step - loss: 0.9885 - accuracy: 0.7914 - val_loss: 0.8460 - val_accuracy: 0.8250
Epoch 6/18
8982/8982 [==============================] - 1s 91us/step - loss: 0.8417 - accuracy: 0.8212 - val_loss: 0.7166 - val_accuracy: 0.8540
Epoch 7/18
8982/8982 [==============================] - 1s 92us/step - loss: 0.72

In [22]:
results

[1.0534891143826004, 0.7898486256599426]

78%의 정확도. 예상대로 유닛 사이즈가 출력해야할 클래스(레이블)의 개수보다 작아지니까 정확도가 감소했다.

In [24]:
# 추가 문제 (128개의 유닛)
# 46차원과 너무 차이가 나니까 결과 더 안좋게 나오지 않을까..?
model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# 컴파일을 한 후에 훈련을 검증해보는 것이다.
# 훈련 데이터 (train)에서 1000개의 샘플을 떼어내서 검증(validation) 세트로 사용한다.
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]


history = model.fit(partial_x_train,partial_y_train,
                    epochs=20, batch_size=512, validation_data=(x_val, y_val))

Train on 7982 samples, validate on 1000 samples
Epoch 1/20
7982/7982 [==============================] - 1s 137us/step - loss: 2.1996 - accuracy: 0.5430 - val_loss: 1.3504 - val_accuracy: 0.6990
Epoch 2/20
7982/7982 [==============================] - 1s 118us/step - loss: 1.0970 - accuracy: 0.7650 - val_loss: 1.0641 - val_accuracy: 0.7850
Epoch 3/20
7982/7982 [==============================] - 1s 126us/step - loss: 0.7763 - accuracy: 0.8359 - val_loss: 0.9474 - val_accuracy: 0.8130
Epoch 4/20
7982/7982 [==============================] - 1s 120us/step - loss: 0.5491 - accuracy: 0.8864 - val_loss: 0.9122 - val_accuracy: 0.8040
Epoch 5/20
7982/7982 [==============================] - 1s 121us/step - loss: 0.4038 - accuracy: 0.9187 - val_loss: 0.8764 - val_accuracy: 0.8130
Epoch 6/20
7982/7982 [==============================] - 1s 121us/step - loss: 0.3158 - accuracy: 0.9312 - val_loss: 0.8498 - val_accuracy: 0.8240
Epoch 7/20
7982/7982 [==============================] - 1s 123us/step - loss

10으로 하자

In [25]:
# 모델 처음부터 다시 훈련
model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# 아까 처음에 훈련할때는 일부만 model.fit(partial_x_train, partial_y_train) 하고
# 과대적합 어딘지 찾은 다음에는 전체인 (x_train, one_hot_train_labels)으로 하는 것이다!!

model.fit(x_train, one_hot_train_labels,
          epochs=10, batch_size=512, validation_data=(x_val, y_val))
results = model.evaluate(x_test, one_hot_test_labels)

Train on 8982 samples, validate on 1000 samples
Epoch 1/10
8982/8982 [==============================] - 1s 136us/step - loss: 2.0768 - accuracy: 0.5794 - val_loss: 1.1489 - val_accuracy: 0.7480
Epoch 2/10
8982/8982 [==============================] - 1s 123us/step - loss: 1.0384 - accuracy: 0.7704 - val_loss: 0.7378 - val_accuracy: 0.8540
Epoch 3/10
8982/8982 [==============================] - 1s 121us/step - loss: 0.7200 - accuracy: 0.8440 - val_loss: 0.4919 - val_accuracy: 0.9030
Epoch 4/10
8982/8982 [==============================] - 1s 121us/step - loss: 0.5159 - accuracy: 0.8940 - val_loss: 0.3740 - val_accuracy: 0.9250
Epoch 5/10
8982/8982 [==============================] - 1s 124us/step - loss: 0.3891 - accuracy: 0.9167 - val_loss: 0.2720 - val_accuracy: 0.9560
Epoch 6/10
8982/8982 [==============================] - 1s 121us/step - loss: 0.2889 - accuracy: 0.9365 - val_loss: 0.2549 - val_accuracy: 0.9500
Epoch 7/10
8982/8982 [==============================] - 1s 119us/step - loss

In [26]:
results

[1.0253722852399492, 0.7978628873825073]

정확도 79%로 감소했다., 예상대로 출력 클래스와 층의 차원이 너무 차이가 나면 정확도가 감소하는 것 같다.